<a href="https://colab.research.google.com/github/sumitag07/sumitag07/blob/main/Reddit_scarpper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt install chromium-chromedriver
!apt install -y xvfb

!pip install undetected-chromedriver
!pip install PyVirtualDisplay

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,081 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,343 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,002 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,267 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.n

In [4]:
!zip -j /content/chromedriver_linux64.zip /usr/bin/chromedriver
#replace python3.7 with your own version of python in case it's not the same
patcher_src = "/content/patcher.py"
with open(patcher_src, "r") as f:
    contents = f.read()
    contents = contents.replace("return urlretrieve(u)[0]",\
                     "return urlretrieve('file:///content/chromedriver_linux64.zip',"\
                     "filename='/tmp/chromedriver_linux64.zip')[0]")
with open(patcher_src, "w") as f:
    f.write(contents)

updating: chromedriver (deflated 49%)


In [5]:
import undetected_chromedriver as uc
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--enable-javascript')
chrome_options.add_argument('--disable-gpu')
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15'
chrome_options.add_argument('User-Agent={0}'.format(user_agent))
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', True)

driver = webdriver.Chrome(options=chrome_options)
#driver.get("https://www.reddit.com/login/")

In [6]:
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin

##### Web scrapper for infinite scrolling page #####
#driver = webdriver.Chrome(executable_path=r"E:\Chromedriver\chromedriver_win32_chrome83\chromedriver.exe")
driver.get("https://www.reddit.com/search/?q=vivek+ramaswamy&type=comment")
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

In [16]:
search_all = []

soup = BeautifulSoup(driver.page_source, "html.parser")
srch = soup.find_all("div", attrs={"class": 'flex justify-between items-center p-md hover:bg-neutral-background-hover xs:rounded-[16px] my-2xs relative'})
#name = ""
n = 0
for result in srch:
  if n < 25:
    name = result.find('div', attrs={'class':'max-h-[260px] overflow-hidden text-ellipsis text-neutral-content-strong hover:no-underline no-underline no-visited'})
    comment_karma = result.find('p', attrs={'class':'text-neutral-content-weak text-12'})
    time.sleep(1)
    search_all.append((name.text,comment_karma.text))
    n = n+1
search_all

[('\n\n    When you read the article, it\'s fucking wild what they bring up. Like, wait, the cultural issues destroying America involve... Yale costume parties no longer accepting racist costumes?!\n  \n\n\n    In the same issue, an article faulted U.S. universities as islands of extremism, where even students\' Halloween costumes are policed, citing Yale University as a place where "offensive" costume-wearers were punished.\n  \n\n\n    Wow - real "fall of an empire" stuff. 🙄\n  \n    Also, apparently it was "woke" for us to leave Afghanistan?\n  \n\n\n    The magazine blamed America\'s retreat from Afghanistan on "a woke dictatorship"\n  \n\n\n    ...I kinda skipped over this the first time I read the piece, but looking at it again, it really jumps out. Yes, the reason the US left a 20-year-long quagmire was because of "wokeness".\n  \n    This is an entirely reasonable point to stop, and say, "Guys, what the actual fuck are you even talking about?" That\'s a statement so bizarrely w

In [18]:
import pandas as pd

df = pd.DataFrame(search_all,columns=['Comment', 'Upvotes'])

In [19]:
df.head()

,Comment,Upvotes
0,"\n\n When you read the article, it's fuckin...",\n\n13\n upvotes\n \n
1,\n\n Looks like a serious thing.\n \n '...,\n\n5\n upvotes\n \n
2,"\n\n\n ""The two most toxic ideologies of th...",\n\n5\n upvotes\n \n
3,"\n\n\n In June Vivek Ramaswamy, a 29-year-o...",\n\n6\n upvotes\n \n
4,\n\n Vivek Ramaswamy is unfathomably based....,\n\n23\n upvotes\n \n


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  25 non-null     object
 1   Upvotes  25 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes


In [21]:
df["Comment"] = df["Comment"].str.replace('\n', '')
df["Upvotes"] = df["Upvotes"].str.replace('\n', '')
df["Upvotes"] = df["Upvotes"].str.replace(' upvotes', '')

In [22]:
df.head()

,Comment,Upvotes
0,"When you read the article, it's fucking wi...",13
1,Looks like a serious thing. 'Founded ...,5
2,"""The two most toxic ideologies of the 20th...",5
3,"In June Vivek Ramaswamy, a 29-year-old for...",6
4,Vivek Ramaswamy is unfathomably based.,23


In [25]:
df.to_csv('Reddit_VR_cmmnt.csv', index=False, encoding='utf-8')